In [5]:
%load_ext autoreload
%autoreload 2

# Create mllib pipeline
from dspipes import Pipelines, MllibPipelines, RumblePipelines

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Write rumble query scripts

In [4]:
MODELS = ['logistic','RandomForest', 'LinearSVC', 'NB']
ALL_PIPELINES = [0, 1, 3, 5]
DATASETS = ['Criteo','YFCC']
for d in DATASETS:
    for i in ALL_PIPELINES:
        for m in MODELS:
            print(d, i, m)
            program = RumblePipelines.create_rumble_program(f"pipe_{i}", clf_mode=f"{m}", dataset=f"{d}")
            f = open(f"./rumbleML_scripts/query_{i}_{m}_{d}.rumble", "w")
            f.write(program)
            f.close()

Criteo 0 logistic
Criteo 0 RandomForest
Hi!
Criteo 0 LinearSVC
Criteo 0 NB
Criteo 1 logistic
Criteo 1 RandomForest
Hi!
Criteo 1 LinearSVC
Criteo 1 NB
Criteo 3 logistic
Criteo 3 RandomForest
Hi!
Criteo 3 LinearSVC
Criteo 3 NB
Criteo 5 logistic
Criteo 5 RandomForest
Hi!
Criteo 5 LinearSVC
Criteo 5 NB
YFCC 0 logistic
YFCC 0 RandomForest
Hi!
YFCC 0 LinearSVC
YFCC 0 NB
YFCC 1 logistic
YFCC 1 RandomForest
Hi!
YFCC 1 LinearSVC
YFCC 1 NB
YFCC 3 logistic
YFCC 3 RandomForest
Hi!
YFCC 3 LinearSVC
YFCC 3 NB
YFCC 5 logistic
YFCC 5 RandomForest
Hi!
YFCC 5 LinearSVC
YFCC 5 NB


In [2]:
RumblePipelines.create_rumble_program(f"pipe_0", clf_mode="RandomForest", dataset=f"Criteo")


Hi!


'let $training-data := parquet-file("s3://rumbleml-data/output/output.parquet/")\nlet $test-data := parquet-file("s3://rumbleml-data/criteo.kaggle2014.test.parquet")\nlet $vector-assembler_2 := get-transformer("VectorAssembler", {"inputCols" : ["features"], "outputCol" : "transformedFeatures"})\nlet $randomforest := get-estimator("RandomForestClassifier", {"featuresCol": "transformedFeatures", "numTrees": 5})\nlet $pipeline := get-estimator("Pipeline", {"stages": [$vector-assembler_2, $randomforest]})\nlet $pip := $pipeline($training-data, {})\nlet $prediction := $pip($test-data, {})\nlet $total := 6042135\nreturn count($prediction[$$.label eq $$.prediction]) div $total'

### Create all experiment bash script

In [7]:
MODELS = ['logistic', 'RandomForest', 'LinearSVC', 'NB']
ALL_PIPELINES = [0, 1, 3, 5]
DATASETS = ['Criteo', 'YFCC']
f = open(f"./run_all_experiments.sh", "a")
for d in DATASETS:
    for i in ALL_PIPELINES:
        for m in MODELS:
            res = f"echo 'Experiment {i} {m} {d}'\ntime spark-submit --num-executors 4 --executor-cores 2 --executor-memory 19g s3://rumbleml-data/rumble_experiments/rumbledb-1.16.0.jar --query-path 's3://rumbleml-data/rumble_experiments/query_{i}_{m}_{d}.rumble'\ntime spark-submit --num-executors 4 --executor-cores 2 --executor-memory 19g run_spark.py -m {m} -p {i} -d {d}\n"
            f.write(res)
f.close() 
